### 语言模型

学习目标

- 学习语言模型，以及如何训练一个语言模型
- 学习torchtex的基本使用方法
    - 构建vocabulary
    - word to index 和 index to word
- 学习torch.nn的一些基本模型
    - Linear
    - RNN
    - LSTM
    - GRU
- RNN的训练技巧
    - Gradient Clipping
- 如何保存和读取模型




![什么是语言模型](images/4.png)

![链式法则](images/5.png)

Markov假设：每个单词只跟它之前n个单词有关
![Markov假设](images/6.png)

语言模型的评价：
句子概率越大，语言模型越好，迷惑度越小。
负号：概率非常小，往往是0.00000...，取负的次方可以变成一个比较大的数字。
1/N：句子的长度是不一样的，长的句子自然概率就小，取1/N次方后可以normalize，把语句的长度的因素取消。
这样就会拿到一个正的数字，PP越低，表示这个语句比较符合模型的预期。
![Perplexity](images/7.png)

用神经网络来拟合概率p
![](images/8.png)

循环神经网络
![](images/9.png)
相当于不断做线性变换，拿到h_t后，要映射到整个单词的空间上去。h_t
可能是300维，y_t是要展示在整个vocab_size，假设是50000维，所以W_s
是一个50000x300的向量
![](images/10.png)

需要在每一个单词上做cross Entropy的计算，y帽是预测结果，y是ground truth
![](images/11.png)

梯度消失和爆炸
![](images/12.png)
梯度太大，往下卡，卡到低于threshold的位置
梯度太小，用relu函数
![](images/13.png)

LSTM
![](images/14.png)

![](images/15.png)
![](images/16.png)
![](images/17.png)
![](images/18.png)
forget gate
![](images/19.png)
i是input gate
![](images/20.png)
![](images/21.png)
output gate
![](images/22.png)
公式，后面的输出对前面求导时，梯度并没有完全小时
![](images/23.png)
更常用的版本，pytorch中使用的
![](images/24.png)
GRU
![](images/25.png)



### RNN
循环神经网络（Recurrent Neural Network，RNN）是一种用于处理序列数据的神经网络。相比一般的神经网络来说，他能够处理序列变化的数据。比如某个单词的意思会因为上文提到的内容不同而有不同的含义，RNN就能够很好地解决这类问题。
![](images/26.png)
x为当前状态下数据的输入，h表示接收到的上一个节点的输入。
y为当前节点状态下的输出，而h’为传递到下一个节点的输出。
从公式可以看出，h'和x、h都相关。y通常使用h'投入到一个线形层（主要是进行维度映射）然后使用softmax进行分类得到所需要的数据。
通过序列形式的输入，可以得到如下形式的RNN：
![](images/27.png)

### LSTM

#### 什么是LSTM
长短期记忆（Long short-term memory, LSTM）是一种特殊的RNN，主要是为了解决长序列训练过程中的梯度消失和梯度爆炸问题。简单来说，就是相比普通的RNN，LSTM能够在更长的序列中有更好的表现。

LSTM结构和普通RNN的主要输入输出区别如下所示。
![](images/28.png)
相比RNN只有一个传递状态h_t，LSTM有两个传输状态，一个c_t（cell state)，和一个h_t(hidden state)。（RNN中的h_t相当于LSTM中的c_t）
其中对于传递下去的c_t改变的很慢，通常输出的c_t是上一个状态传过来的c_(t-1)加上一些数值。

#### 深入LSTM结构
首先使用LSTM的当前输入x_t和上一个状态传递下来的h_(t-1)拼接训练得到四个状态。
![](images/29.png)
其中，z_f,z_i,z_o是由拼接向量乘以权重矩阵之后，再通过一个sigmoid激活函数转换成0到1之间的数值，来作为一种门控状态。而z则是将结果通过一个tanh激活函数转换到（-1，1）之间的值（这里使用tanh是因为这里是将其作为输入数据，而不是门控信号）。
![](images/30.png)
LSTM内部主要有三个阶段
- 忘记阶段。这个阶段主要是对上一个节点传进来的输入进行选择性忘记。简单来说就是会“忘记不重要的，记住重要的”。具体来说是通过计算得到的z_f（f表示forget）来作为忘记门控，来控制上一个状态的c_(t-1)哪些需要留哪些需要忘。
- 选择记忆阶段。这个阶段将这个阶段的输入有选择性地进行“记忆”。主要是会对输入x_t进行选择记忆。哪些重要则着重记录下来，哪些不重要，则少记一些。当前的输入内容由前面计算得到的z表示，而选择的门控信号则是由z_i(i代表information)来进行控制。
    - 将上面两步得到的结果相加，即可得到传输给下一个状态的c_t。也就是上图中的第一个公式。
- 输出阶段。这个阶段将决定哪些将会被当成当前状态的输出。主要是通过z_o来进心控制的。并且还对上一阶段得到的c_o进行了放缩(通过一个tanh激活函数来进行变化)。

与普通RNN类似，输出y_t往往最终也是通过h_t变化得到的。

#### 总结
以上，就是LSTM的内部结构。通过门控状态来控制传输状态，记住需要长时间记忆的，忘记不重要的信息；而不像普通的RNN那样仅有一种记忆叠加方式。对很多需要“长期记忆”的任务来说，尤其好用。

但也因为引入了很多内容，导致参数变多，也使得训练难度加大了很多。因此很多时候我们往往会使用效果和LSTM相当但参数更少的GRU来构建大训练量的模型。

### GRU
GRU是RNN的一种，和LSTM一样，是为了解决长期记忆和反向传播中的梯度等问题而提出来的。
GRU和LSTM在很多情况下实际表现上相差无几，那么为什么要使用较新的GRU（2014）而不是相对经受了更多考验的LSTM（1997）呢？原因就是GRU的实验效果与LSTM相似，但是更容易计算。相比LSTM，使用GRU能够达到相当的效果，并且相比之下更容易进行训练，能够很大程度上提高训练效率，因此很多时候会更倾向于使用GRU。
#### GRU的输入输出结构
GRU的输入输出结构和普通的RNN是一样的。
有一个当前的输入x_t，和上一个节点传递下来的隐状态h_(t-1)，这个隐状态包含了之前节点的相关信息。
结合x_t和h_(t-1)，GRU就会得到当前隐藏节点的输出y_t和传递给下一个节点的隐状态h_t。
![GRU的输入输出结构](images/31.png)

#### GRU的内部结构
首先，我们先通过上一个传输下来的状态h_(t-1)和当前节点的输入x_t来获取两个门控状态。如下图所以，其中r控制重置的门控（reset gate），z为控制更新的门控（update gate）。
![GRU的输入输出结构](images/32.png)
得到门控信号之后，首先使用重置门来得到“重置”之后的数据$h^{t-1}$$'$=$h^{t-1}\bigodot r$,再将h_(t-1)'与输入x_t进行拼接，再通过一个tanh激活函数来将数据放缩到(-1,1)的范围内。即得到如下所示的h'。
![](images/33.png)
这里的h'主要是包含了当前输入的x_t数据，有针对性地对h'添加到当前的隐藏状态，相当于记忆了当前时刻的状态，类似于LSTM地选择记忆阶段。

![](images/33.png)

最后介绍GRU最关键的一个步骤，可以称之为“更新记忆”阶段。
在这个阶段，我们同时进行了遗忘了记忆两个步骤。我们使用了先前得到的更新门控z（update gate）。

更新表达式：$h^t$ = $(1-z) \bigodot h^{t-1} + z \bigodot h'$

首先再次强调一下，门控信号（z）的范围为0-1，门控信号越接近1，代表“记忆”下来的数据越多；而越接近0，则代表“遗忘”的越多。

**GRU很聪明的一点就在于，我们使用了同一个门控z就同时可以进行遗忘和选择记忆（LSTM则要使用多个门控）。**

- $(1-z) \bigodot h^{t-1}$ ：表示对原本隐藏状态的选择性“遗忘”。这里的1-z可以想象成遗忘门，忘记h_(t-1)维度中一些不重要的信息。
- $z \bigodot h'$ ：表示对包含当前节点信息的h’进行选择性“记忆”。与上面类似，这里的（1-z）同理会忘记h’维度中的一些不重要的信息。或者，这里我们更应当看作是对h’维度中的某些信息进行选择。
- $h^t = (1-z) \bigodot h^{t-1} + z \bigodot h' $：结合上述，这一步的操作就是忘记传递下来的h_(t-1)中的某些维度信息，并加入当前节点输入的某些维度信息。
>可以看到，这里的遗忘z和选择(1-z）是联动的，也就是说，对于传递进来的维度信息，我们会进行选择性遗忘，则遗忘了多少权重（z），我们就会使用包含当前输入的h'中所对应的权重进行弥补(1-z），以保持一种“恒定”状态。

#### LSTM和GRU的关系
在这里，r(reset gate)实际上与他的名字有点不符，我们仅仅用它来获得了h'。
那么这里的h'实际上可以看成对应于LSTM中的hidden state；上一个节点传下来的h_(t-1)则对应于LSTM中的cell state。1-z对应的则是LSTM中的z_f forget gate，那么z似乎就可以看成选择门z_i了。

我们会使用torchtext来创建vocabulary，然后把数据读成batch的格式。

In [1]:
import torchtext
from torchtext.vocab import Vectors
import torch
import numpy as np
import random
import warnings

USE_CUDA = torch.cuda.is_available()

# 为了保证实验结果可以复现，我们经常会把各种random seed固定在某一个值
random.seed(53113)
np.random.seed(53113)
torch.manual_seed(53113)
if USE_CUDA:
    torch.cuda.manual_seed(53113)

BATCH_SIZE = 32
EMBEDDING_SIZE = 100
MAX_VOCAB_SIZE = 50000
HIDDEN_SIZE = 100

warnings.filterwarnings("ignore")

- 我们会继续使用上次的text8作为我们的训练、验证和测试数据
- TorchText的一个重要概念是Field，它决定了你的数据会如何被处理。我们使用TEXT这个field来处理文本数据。我们的TEXT field有lower =True这个参数，所以所有的单词都会被lowercase。
- torchtext提供了LanguageModelingDatset这个class来帮助我们处理语言模型数据集。
- build——vocab可以根据我们提供的训练数据集来创建最高频单词的单词表，max_size帮助我们限定单词总量
- BPTTIterator可以连续地得到连贯的句子，BPTT的全称是back propagation through time

In [2]:
TEXT = torchtext.data.Field(lower=True)
train, val, test = torchtext.datasets.LanguageModelingDataset.splits(path=".",
            train = "text8/text8.train.txt",validation = "text8/text8.dev.txt",
            test = "text8/text8.test.txt",text_field=TEXT)



In [3]:
# 选取频率最高的单词留下来，作为单词表，torchtext做好了这件事
TEXT.build_vocab(train, max_size=MAX_VOCAB_SIZE)


In [4]:
len(TEXT.vocab)

50002

- 为什么我们的单词表有50002个单词而不是50000呢？因为Torch Text给我们增加了两个特殊的token，```<unk>```表示未知的单词，```<pad>```表示padding(补全太短的单词)
- 模型的输入是一串文字，模型的输出也是一串文字，他们之间相差一个位置，因为语言模型的目标是根据之前的单词预测下一个单词。

In [5]:
TEXT.vocab.itos[:10]

['<unk>', '<pad>', 'the', 'of', 'and', 'one', 'in', 'a', 'to', 'zero']

In [6]:
TEXT.vocab.stoi["apple"]

1273

In [7]:
device = torch.device("cuda" if USE_CUDA else "cpu")
device

device(type='cuda')

In [8]:
# 现在要构建一个iterator，在训练和测试集上有这么多的text，希望把它变成一个个的batch
# 每个batch里面包含32个句子，用这个32个句子作训练
# bptt_len是往回传的长度要有多少
# repeat走往一个文件后会不会重复
train_iter, val_iter, test_iter = torchtext.data.BPTTIterator.splits(
        (train, val, test), batch_size=BATCH_SIZE, device=device,
        bptt_len=50, repeat=False, shuffle=True)

In [9]:
it = iter(train_iter)
batch = next(it)

In [10]:
# text是文本
# target是要预测的句子
# 50是句子的长度，32是batch_size
batch


[torchtext.data.batch.Batch of size 32]
	[.text]:[torch.cuda.LongTensor of size 50x32 (GPU 0)]
	[.target]:[torch.cuda.LongTensor of size 50x32 (GPU 0)]

In [11]:
batch.text

tensor([[4815,   50,    6,  ..., 9116,   33,    7],
        [3143, 2748,  495,  ...,  893,  277,  317],
        [  13,    8,  850,  ...,  664,  824, 1602],
        ...,
        [   8,   34,  522,  ..., 5237,    3,   12],
        [3628, 1266,  968,  ...,    3,    2,    6],
        [   2,   54,   78,  ...,   12,  185, 3027]], device='cuda:0')

In [12]:
print(" ".join(TEXT.vocab.itos[i] for i in batch.text[:,0].data.cpu()))
print()
print(" ".join(TEXT.vocab.itos[i] for i in batch.target[:,0].data.cpu()))

anarchism originated as a term of abuse first used against early working class radicals including the diggers of the english revolution and the sans <unk> of the french revolution whilst the term is still used in a pejorative way to describe any act that used violent means to destroy the

originated as a term of abuse first used against early working class radicals including the diggers of the english revolution and the sans <unk> of the french revolution whilst the term is still used in a pejorative way to describe any act that used violent means to destroy the organization


In [13]:
for i in range(5):
    batch = next(it)
    print(i)
    print(" ".join(TEXT.vocab.itos[i] for i in batch.text[:,0].data.cpu()))
    print()
    print(" ".join(TEXT.vocab.itos[i] for i in batch.target[:,0].data.cpu()))

0
organization of society it has also been taken up as a positive label by self defined anarchists the word anarchism is derived from the greek without archons ruler chief king anarchism as a political philosophy is the belief that rulers are unnecessary and should be abolished although there are differing

of society it has also been taken up as a positive label by self defined anarchists the word anarchism is derived from the greek without archons ruler chief king anarchism as a political philosophy is the belief that rulers are unnecessary and should be abolished although there are differing interpretations
1
interpretations of what this means anarchism also refers to related social movements that advocate the elimination of authoritarian institutions particularly the state the word anarchy as most anarchists use it does not imply chaos nihilism or <unk> but rather a harmonious anti authoritarian society in place of what are regarded

of what this means anarchism also refers to rela

### 定义模型
- 继承nn.Module
- 初始化函数
- forward函数
- 其余可以根据模型需要定义相关的函数

In [28]:
import torch.nn as nn

class RNNModel(nn.Module):
    # 以LSTM为例
    def __init__(self, vocab_size, embed_size, hidden_size):
        # 初始化继承的类
        super(RNNModel, self).__init__()
        # 把每个单词映射成embed_size大小的向量,随机初始化，后面会训练
        self.embed = nn.Embedding(vocab_size, embed_size)
        
        # input_size:输入数据的特征维数，通常就是embedding_dim（词向量的维度
        # hidden_size:LSTM中隐层的维度
        # num_layers: 循环神经网络的层数
        # num_directions=2,否则就是1，表示只有1个方向
        self.lstm = nn.LSTM(embed_size, hidden_size)
        
        # 最后输出要是一个50002维的向量，才能被argmax
        self.linear = nn.Linear(hidden_size, vocab_size)
        self.hidden_size = hidden_size
        
    def forward(self, text, hidden):
        
        # text:seq_length * batch_size
        emb = self.embed(text) # seq_length * batch_size * embed_size
        output, hidden = self.lstm(emb, hidden) 
        # output:seq_length * batch_size * hidden_size 把输出的每一个hidden_states都拿出来了
        # (lstm可以有若干层，1层的话就是1*）hidden: (1 * batchsize * hidden_size,  1 * batch_size * hidden_size)
        # hidden: 最后一个hidden_states
        # output[-1]与h_n是相等的，因为output[-1]包含的正是batch_size个句子中每一个句子的最后一个单词的隐藏状态，注意LSTM中的隐藏状态其实就是输出，cell state细胞状态才是LSTM中一直隐藏的，记录着信息
        
        
        # output是三维的，但是做线性变换的时候要求output是两维的
        # 把前两个维度拼到一起
        # output = output.view(-1, output.shape[2]) # (seq_length * batch_size) * hidden_size 
        out_vocab = self.linear(output.view(-1, output.shape[2])) # (seq_length * batch_size) * vocab_size
        out_vocab = out_vocab.view(output.size(0), output.size(1), out_vocab.size(-1))
        
        #每一个位置分别预测了哪一个单词
        return out_vocab, hidden
        
    
    def init_hidden(self, bsz, requires_grad = True):
        # 初始化hidden_state
        weight = next(self.parameters())
        return (weight.new_zeros((1, bsz, self.hidden_size), requires_grad=True),
                weight.new_zeros((1, bsz, self.hidden_size), requires_grad=True))
        
        
        
        

初始化一个模型

In [29]:
model = RNNModel(vocab_size=len(TEXT.vocab), 
                 embed_size=EMBEDDING_SIZE,
                 hidden_size=HIDDEN_SIZE)
if USE_CUDA:
    model = model.to(device)


In [30]:
model

RNNModel(
  (embed): Embedding(50002, 100)
  (lstm): LSTM(100, 100)
  (linear): Linear(in_features=100, out_features=50002, bias=True)
)

In [31]:
def repackage_hidden(h):
    # 把这个h和之前所有的history全部detach掉
    # 把这个节点从前面的计算图里截断，等于这个h是复制了原来的值，但没有把历史背下来
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        # h如果是两个，每个都package一下
        return tuple(repackage_hidden(v) for v in h)

训练模型

In [35]:
loss_fn = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# scheduler可以帮助调lr
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.5)

In [33]:
VOCAB_SIZE = len(TEXT.vocab)

In [46]:
def evaluate(model, data):
    model.eval() # 训练模式
    total_loss = 0.
    total_count = 0.
    it = iter(data)
    with torch.no_grad():
        hidden = model.init_hidden(BATCH_SIZE, requires_grad=False)
        for i, batch in enumerate(it):
            data, target = batch.text, batch.target

            # 确保是全新的hidden，而不是带着一大堆历史的hidden
            hidden = repackage_hidden(hidden)
            # 因为句子首尾相连，所以可以直接传hidden
            output, hidden = model(data, hidden) # backpropgate through all iterations 计算图会非常大而且非常深

            loss = loss_fn(output.view(-1, VOCAB_SIZE), target.view(-1)) # batch_size * target_class_dim, batch_size
            total_loss = loss.item() * np.multiply(*data.size())
            total_count = np.multiply(*data.size())
    
    loss = total_loss / total_count
    model.train()
    return loss

In [47]:
NUM_EPOCHS = 2
GRAD_CLIP = 5.

val_losses = []
for epoch in range(NUM_EPOCHS):
    model.train() # 训练模式
    it = iter(train_iter)
    hidden = model.init_hidden(BATCH_SIZE)
    for i, batch in enumerate(it):
        data, target = batch.text, batch.target
        
        # 确保是全新的hidden，而不是带着一大堆历史的hidden
        hidden = repackage_hidden(hidden)
        # 因为句子首尾相连，所以可以直接传hidden
        output, hidden = model(data, hidden) # backpropgate through all iterations 计算图会非常大而且非常深
        
        loss = loss_fn(output.view(-1, VOCAB_SIZE), target.view(-1)) # batch_size * target_class_dim, batch_size
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
        
        optimizer.step()
        
        # 将训练时的模型给存下来
        if i % 100 == 0:
            print("epoch", epoch, "iteration", i, "loss", loss.item())
        
        if i % 1000 == 0:
            val_loss = evaluate(model, val_iter)
            if len(val_losses) == 0 or val_loss < min (val_losses):

                torch.save(model.state_dict(), "lm.pth")
                print("best model saved to lm.pth")
            else:
                # 发现loss降不下来，那么降低lr，lr降低一点会使模型调的更精细
                # learning rate decay
                scheduler.step()lo
            val_losses.append(val_loss)
                
        

epoch 0 iteration 0 loss 7.050442695617676
best model saved to lm.pth
epoch 0 iteration 100 loss 6.808845043182373
epoch 0 iteration 200 loss 6.653306484222412
epoch 0 iteration 300 loss 6.944944381713867
epoch 0 iteration 400 loss 6.815479278564453
epoch 0 iteration 500 loss 6.660840034484863
epoch 0 iteration 600 loss 6.570061206817627
epoch 0 iteration 700 loss 6.5757060050964355
epoch 0 iteration 800 loss 6.40944766998291
epoch 0 iteration 900 loss 6.6818437576293945
epoch 0 iteration 1000 loss 6.7329559326171875
best model saved to lm.pth
epoch 0 iteration 1100 loss 6.416387557983398
epoch 0 iteration 1200 loss 6.332454204559326
epoch 0 iteration 1300 loss 6.446369171142578
epoch 0 iteration 1400 loss 6.128020763397217
epoch 0 iteration 1500 loss 6.2821364402771
epoch 0 iteration 1600 loss 6.072458267211914
epoch 0 iteration 1700 loss 6.366618633270264
epoch 0 iteration 1800 loss 6.417938232421875
epoch 0 iteration 1900 loss 6.3945441246032715
epoch 0 iteration 2000 loss 6.3500037

epoch 1 iteration 7300 loss 5.5129876136779785
epoch 1 iteration 7400 loss 5.531670093536377
epoch 1 iteration 7500 loss 5.377089977264404
epoch 1 iteration 7600 loss 5.426165580749512
epoch 1 iteration 7700 loss 5.553062915802002
epoch 1 iteration 7800 loss 5.510672092437744
epoch 1 iteration 7900 loss 5.529897689819336
epoch 1 iteration 8000 loss 5.475377082824707
best model saved to lm.pth
epoch 1 iteration 8100 loss 5.455070972442627
epoch 1 iteration 8200 loss 5.290660381317139
epoch 1 iteration 8300 loss 5.251063823699951
epoch 1 iteration 8400 loss 5.690160751342773
epoch 1 iteration 8500 loss 5.515962600708008
epoch 1 iteration 8600 loss 5.494745254516602
epoch 1 iteration 8700 loss 5.360583305358887
epoch 1 iteration 8800 loss 5.385583400726318
epoch 1 iteration 8900 loss 5.7680253982543945
epoch 1 iteration 9000 loss 5.398598670959473
best model saved to lm.pth
epoch 1 iteration 9100 loss 5.521969795227051
epoch 1 iteration 9200 loss 5.241772651672363
epoch 1 iteration 9300 l

In [48]:
# 把一个模型的参数load回来
best_model = RNNModel(vocab_size=len(TEXT.vocab), 
                 embed_size=EMBEDDING_SIZE,
                 hidden_size=HIDDEN_SIZE)
if USE_CUDA:
    best_model = best_model.to(device)

best_model.load_state_dict(torch.load("lm.pth"))

<All keys matched successfully>

使用训练好的模型生成一些句子

In [56]:
if USE_CUDA:
    best_model = best_model.to(device)
hidden = best_model.init_hidden(1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 随机生成一个单词
input = torch.randint(VOCAB_SIZE, (1, 1), dtype=torch.long).to(device)
words = []
for i in range(100):
    
    # 拿到一个单词
    output, hidden = best_model(input, hidden)
    # 一个50002维的vector 把所有维度为1的维度扔掉 
    # exp:放大概率，让大的更大，小的更小
    word_weights = output.squeeze().exp().cpu()
    # 找到概率最大的单词
    word_idx = torch.multinomial(word_weights, 1)[0] # greedy(argmax)
    input.fill_(word_idx)
    word = TEXT.vocab.itos[word_idx]
    words.append(word)
print(" ".join(words))

fallacious commanded by the dominant majority of decades the easter twelve egyptians see egypt s government although reload their head has consistently provision and converted party jos tapes according to a talent of gory physicist first partially played at university of vienna john edward mann however for revisionists and he would ask the special pot of his captured and based minnesota african people as willard s appointment as to hughes president clinton remained on the church of convicts any deaths served in one nine two five <unk> rotterdam rommel with the last catechetical district k d a roman second republic
